<a href="https://colab.research.google.com/github/JustinPark845/Natural-Language-Processing-Final-Project/blob/main/MergeDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ast
# 2018; threshold = 5 upvotes; limit = none
f=open("/content/drive/MyDrive/nlp_project/5thres_res_2018.txt", "r")
reddit2018 = ast.literal_eval(f.read())
f.close()

# 2019; threshold = 5 upvotes; limit = none
f=open("/content/drive/MyDrive/nlp_project/5thres_res_2019.txt", "r")
reddit2019 = ast.literal_eval(f.read())
f.close()

# 2021; threshold = 5 upvotes; limit = none
f=open("/content/drive/MyDrive/nlp_project/5thres_res_2021.txt", "r")
reddit2021 = ast.literal_eval(f.read())
f.close()

# 2022; threshold = none; limit = 10
f=open("/content/drive/MyDrive/nlp_project/10lim_res_2022.txt", "r")
reddit2022 = ast.literal_eval(f.read())
f.close()

date_to_reddit_post = reddit2018
date_to_reddit_post.update(reddit2019)
date_to_reddit_post.update(reddit2021)
date_to_reddit_post.update(reddit2022)

f=open("/content/drive/MyDrive/nlp_project/yfinance.txt", "r")
yfinance = ast.literal_eval(f.read())
f.close()

In [ ]:
import datetime as dt

text_to_label = []

count = 0
current_date = dt.date(2018, 1, 1)
end_date = dt.date.today()
while (current_date <= end_date):
  print(current_date, len(date_to_reddit_post.get(str(current_date),[])), date_to_reddit_post.get(str(current_date)))
  date_posts = date_to_reddit_post.get(str(current_date),False)
  if date_posts:
    for title, text_entry in date_posts:
      for ticker in ["AMZN","AAPL","MSFT","GOOGL","TSLA","NVDA","AMD","NFLX","SPY","META"]:
        if title.upper().find(ticker) != -1:
          if current_date.weekday() == 4:
            if current_date.year in [2018, 2019]:
              label = yfinance[ticker][0].get(str(current_date + dt.timedelta(days=3)),None)
            else:
              label = yfinance[ticker][1].get(str(current_date + dt.timedelta(days=3)),None)
          elif current_date.weekday() == 5:
            if current_date.year in [2018, 2019]:
              label = yfinance[ticker][0].get(str(current_date + dt.timedelta(days=2)),None)
            else:
              label = yfinance[ticker][1].get(str(current_date + dt.timedelta(days=2)),None)
          else:
            if current_date.year in [2018, 2019]:
              label = yfinance[ticker][0].get(str(current_date + dt.timedelta(days=1)),None)
            else:
              label = yfinance[ticker][1].get(str(current_date + dt.timedelta(days=1)),None)
          if label:
            count += 1
            text_to_label.append({"text": text_entry, "label": label})
  current_date += dt.timedelta(days=1)

In [ ]:
import random
random.seed(41)

random.shuffle(text_to_label)
text_to_label_train = text_to_label[:round(len(text_to_label) * 0.8)]
text_to_label_test = text_to_label[round(len(text_to_label) * 0.8):]

In [ ]:
# Create txt
f=open("/content/drive/MyDrive/nlp_project/text_to_label.txt", "w")
f.write(str(text_to_label))
f.close()

In [ ]:
# Create csv
import csv  

header = ['text','label']

with open('/content/drive/MyDrive/nlp_project/text_to_label_train.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in text_to_label_train:
      if i["label"] < 0:
        writer.writerow([i["text"],0])
      else:
        writer.writerow([i["text"],i["label"]])

with open('/content/drive/MyDrive/nlp_project/text_to_label_test.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    for i in text_to_label_test:
      if i["label"] < 0:
        writer.writerow([i["text"],0])
      else:
        writer.writerow([i["text"],i["label"]])